In [6]:
import numpy as np
from scipy.spatial.transform import Rotation

m = 60
X = np.zeros((m, 3))  
np.random.seed(42)
angles = (np.random.rand(m) ** 3 + 0.5) * 2 * np.pi  
X[:, 0], X[:, 1] = np.cos(angles), np.sin(angles) * 0.5  # oval
X += 0.28 * np.random.randn(m, 3)  # add more noise
X = Rotation.from_rotvec([np.pi / 29, -np.pi / 20, np.pi / 4]).apply(X)
X += [0.2, 0, 0.2]

In [7]:
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt[0]
c2 = Vt[1]
print(c1)
print(c2)


[0.67857588 0.70073508 0.22023881]
[-0.72817329  0.6811147   0.07646185]


In [8]:
W2 = Vt[:2].T
X2D = X_centered @ W2
X2D

array([[-8.73231190e-01, -2.94598030e-01],
       [ 1.48885182e-01,  5.14935573e-01],
       [ 1.35121872e+00, -3.99501548e-01],
       [ 4.54366763e-01, -1.39984497e-01],
       [-7.34389086e-01, -2.28934648e-02],
       [-2.33347464e-01, -8.67844755e-02],
       [-8.08435321e-01, -2.52457557e-01],
       [ 1.48281454e+00,  4.22796305e-01],
       [ 3.85679006e-01, -6.16229365e-01],
       [ 1.54972180e+00, -3.60381563e-01],
       [-1.22938760e+00,  3.12504780e-01],
       [-4.54653275e-01,  3.28839370e-01],
       [ 1.34315899e+00,  3.79446240e-01],
       [-6.55233341e-01, -1.92367174e-01],
       [-2.49510114e-01,  1.28486810e-01],
       [-3.46562831e-01,  8.32312189e-02],
       [-6.90221113e-01, -1.13712645e-01],
       [-5.29757591e-01, -2.40403321e-01],
       [-3.96344855e-01, -2.60334107e-01],
       [-6.19519220e-01, -1.13588889e-01],
       [ 3.34910399e-01, -3.09476565e-01],
       [-4.52441114e-01,  1.28501562e-01],
       [-1.02718730e+00, -7.20555799e-03],
       [ 2.

In [9]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [10]:
pca.components_

array([[ 0.67857588,  0.70073508,  0.22023881],
       [ 0.72817329, -0.6811147 , -0.07646185]])

In [11]:
pca.explained_variance_ratio_

array([0.7578477 , 0.15186921])

In [12]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)
X_train, y_train = mnist.data[:60_000], mnist.target[:60_000]
X_test, y_test = mnist.data[60_000:], mnist.target[60_000:]

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

e:\Python\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [13]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)
pca.n_components_

154

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

clf = make_pipeline(PCA(random_state=42),
                    RandomForestClassifier(random_state=42))

param_distrib = {
                    "pca__n_components": np.arange(10, 80),
                    "randomforestclassifier__n_estimators": np.arange(50, 500)
}
rnd_search = RandomizedSearchCV(clf, param_distrib, n_iter=10, cv=3,
                                random_state=42)
rnd_search.fit(X_train[:1000], y_train[:1000])


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'pca__n_components': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       6...
       414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478,
       479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
       492, 493, 494, 495, 496, 497, 498, 499])},
                   random_state=42)

In [15]:
print(rnd_search.best_params_)

{'randomforestclassifier__n_estimators': 465, 'pca__n_components': 23}


In [16]:
X_recovered = pca.inverse_transform(X_reduced)
X_recovered

array([[ 2.64687454e-14, -8.06472700e-13, -1.65104982e-13, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.64609603e-15, -5.51507399e-14,  1.62443434e-13, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.28377776e-15,  4.88971987e-14, -2.66207908e-13, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 8.88183138e-15, -4.12823352e-13,  3.66639180e-14, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.07364675e-16,  5.74281168e-14, -8.69618948e-14, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.73667409e-15, -1.38777459e-13,  1.34765580e-13, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [17]:
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)
X_reduced


array([[ 123.93258864, -312.67426198,  -24.51405174, ...,  -62.00213296,
          -8.8147422 ,  -66.93993166],
       [1011.71837586, -294.85703831,  596.33956108, ...,  -24.52514836,
          26.58534428,   16.99077095],
       [ -51.84960804,  392.17315289, -188.50974941, ...,   -8.99144972,
          -2.99473092,   56.93622984],
       ...,
       [-178.0534496 ,  160.0782111 , -257.61308227, ...,   35.30439525,
          -2.75142691,   23.97581712],
       [ 130.60607212,   -5.59193632,  513.85867376, ...,  -15.84132904,
         -18.38612585,   39.40742042],
       [-173.43595246,  -24.71880228,  556.01889398, ...,   29.62816702,
         -52.61652274,   27.99524134]])

In [18]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)

for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_reduced = inc_pca.transform(X_train)


In [ ]:
filename = "my_mnist.mmap"
X_mmap = np.memmap(filename, dtype='float32', mode='write',
                    shape=X_train.shape)

X_mmap[:] = X_train 
X_mmap.flush()

In [ ]:
X_mmap = np.memmap(filename, dtype="float32", mode="readonly").reshape(-1,784)

batch_size = X_mmap.shape[0] // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)

inc_pca.fit(X_mmap)


In [ ]:
from sklearn.random_projection import johnson_lindenstrauss_min_dim
m, ε = 5_000, 0.1
d = johnson_lindenstrauss_min_dim(m, eps=ε)
d

In [ ]:
n = 20_000
np.random.seed(42)
P = np.random.randn(d, n) / np.sqrt(d)
X = np.random.randn(m, n) 
X_reduced = X @ P.T
X_reduced

In [ ]:
from sklearn.random_projection import GaussianRandomProjection

gaussian_rnd_proj = GaussianRandomProjection(eps=ε, random_state=42)
X_reduced = gaussian_rnd_proj.fit_transform(X) 

In [ ]:
components_pinv = np.linalg.pinv(gaussian_rnd_proj.components_)
X_recovered = X_reduced @ components_pinv.T
X_recovered


In [ ]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_unrolled = lle.fit_transform(X_swiss)
X_unrolled
